In [2]:
import warnings
import numpy as np
import pandas as pd
from pathlib import Path
warnings.filterwarnings('ignore')


base_path = Path("./digital")
# 当年发布数字相关职位的总条数 + 当年发布数字相关职位的招聘人数 
dataset = pd.read_excel(base_path/"数字人力投入计划统计表.xlsx", skiprows=[0, 2], dtype={'证券代码': 'object'}, engine="openpyxl")
dataset["统计年度"] = dataset["统计月度"].apply(lambda x: x.split('-')[0].strip())
dataset["统计年度"] = dataset["统计年度"].astype("int")
dataset.fillna(0, inplace=True)
dataset = dataset.groupby(['证券代码', "统计年度"]).sum().reset_index()[['证券代码', "统计年度", "发布需求次数", "招聘人数"]]
# 数字创新标准工作
temp = pd.read_excel(base_path/'数字创新标准工作统计表.xlsx', skiprows=[0, 2], dtype={'证券代码': 'object'}, engine="openpyxl")
temp["统计年度"] = temp["统计年度"].astype("int")
temp.fillna(0, inplace=True)
dataset = pd.merge(dataset, temp[["统计年度","证券代码", "当年参与国家标准制定数量","当年参与行业标准制定数量"]], on=["统计年度","证券代码"],  how="outer")
# 数字创新资质
temp = pd.read_excel(base_path/'数字创新资质统计表.xlsx', skiprows=[0, 2], dtype={'证券代码': 'object'}, engine="openpyxl")
temp["统计年度"] = temp["统计年度"].astype("int")
temp.rename({"数量":"数字创新资质认定数量"},axis=1, inplace=True)
temp.fillna(0, inplace=True)
dataset = pd.merge(dataset, temp[["统计年度","证券代码", "数字创新资质认定数量"]], on=["统计年度","证券代码"],  how="outer")
# 数字发明专利授权
temp = pd.read_excel(base_path/'数字发明专利授权情况表.xlsx', skiprows=[0, 2], dtype={'证券代码': 'object'}, engine="openpyxl")
temp["统计年度"] = temp["统计年度"].astype("int")
temp.fillna(0, inplace=True)
dataset = pd.merge(dataset, temp[["统计年度","证券代码", "数字发明专利授权量"]], on=["统计年度","证券代码"],  how="outer")
# 科技创新基地建设
temp = pd.read_excel(base_path/'科技创新基地建设情况表.xlsx', skiprows=[0, 2], dtype={'证券代码': 'object'}, engine="openpyxl")
temp["统计年度"] = temp["统计年度"].astype("int")
temp.fillna(0, inplace=True)
dataset = pd.merge(dataset, temp[["统计年度","证券代码", "累计参与国家级科技创新基地建设数量"]], on=["统计年度","证券代码"],  how="outer")
# 数字国家级奖项
temp = pd.read_excel(base_path/'数字国家级奖项统计表.xlsx', skiprows=[0, 2], dtype={'证券代码': 'object'}, engine="openpyxl")
temp["统计年度"] = temp["统计年度"].astype("int")
temp.rename({"奖项数量":"数字化国家级奖项数量"},axis=1, inplace=True)
temp.fillna(0, inplace=True)
dataset = pd.merge(dataset, temp[["统计年度","证券代码", "数字化国家级奖项数量"]], on=["统计年度","证券代码"],  how="outer")
# 管理层数字职务设立
temp = pd.read_excel(base_path/'管理层数字职务设立情况表.xlsx', skiprows=[0, 2], dtype={'证券代码': 'object'}, engine="openpyxl")
temp["统计年度"] = temp["统计年度"].astype("int")
temp.fillna(0, inplace=True)
temp["管理层数字职务设立数"] = temp["是否设置首席信息官（CIO）"]+temp["是否设置首席数字官（CDO）"] + temp["是否设置首席技术官（CTO）"]*2
temp.fillna(0, inplace=True)
dataset = pd.merge(dataset, temp[["统计年度","证券代码", "管理层数字职务设立数"]], on=["统计年度","证券代码"],  how="outer")
# 管理层数字创新导向指标
temp = pd.read_excel(base_path/'管理层数字创新导向指标表.xlsx', skiprows=[0, 2], dtype={'证券代码': 'object'}, engine="openpyxl")
temp["统计年度"] = temp["统计年度"].astype("int")
temp["创新导向前瞻性"] = temp.apply(lambda row: row['统计年度'] - row['创新导向前瞻性'], axis=1)
temp.fillna(0, inplace=True)
dataset = pd.merge(dataset, temp[["统计年度","证券代码", "创新导向前瞻性","创新导向持续性","创新导向广度","创新导向强度"]], on=["统计年度","证券代码"],  how="outer")
# 数字资本投入计划
temp = pd.read_excel(base_path/'数字资本投入计划统计表.xlsx', skiprows=[0, 2], dtype={'证券代码': 'object'}, engine="openpyxl")
temp["统计年度"] = temp["统计年度"].astype("int")
temp.fillna(0, inplace=True)
dataset = pd.merge(dataset, temp[["统计年度","证券代码", "计划投资项目数","计划投资总金额"]], on=["统计年度","证券代码"],  how="outer")
# 数字创新论文发表数量
temp = pd.read_excel(base_path/'数字创新论文数量统计表.xlsx', skiprows=[0, 2], dtype={'证券代码': 'object'}, engine="openpyxl")
temp["统计年度"] = temp["统计年度"].astype("int")
temp["数字创新论文发表数量"] = temp["国内期刊发表论文数量"] + temp["国外期刊发表论文数量"]
temp.fillna(0, inplace=True)
dataset = pd.merge(dataset, temp[["统计年度","证券代码", "数字创新论文发表数量"]], on=["统计年度","证券代码"],  how="outer")
csr = pd.read_excel("上市公司社会责任报告基本信息表.xlsx" , skiprows=[0, 2], dtype={'股票代码': 'object'}, parse_dates=['统计截止日期'], engine="openpyxl")
csr['截止日期'] = csr['统计截止日期'].dt.year.astype('int')
csr['公告披露意愿'] = csr['公告披露意愿'].replace({1: 2, 2: 1})
_inds = ['纳税总额',  '每股社会贡献值', '社会捐赠额', '是否经第三方机构审验',  '是否参照GRI《可持续发展报告指南》', '是否披露股东权益保护', '是否披露债权人权益保护', '是否披露职工权益保护', '是否披露供应商权益保护', '是否披露客户及消费者权益保护', '是否披露环境和可持续发展', '是否披露公共关系和社会公益事业', '是否披露社会责任制度建设及改善措施', '是否披露安全生产内容', '是否披露公司存在的不足', '公告披露意愿', '审计师是否来自四大会计师事务所']
csr = csr[(csr["数据来源"]==1)][["股票代码", '截止日期', *_inds]]
csr.replace({2: 0, np.nan: 0.0}, inplace=True)
csr.rename({"股票代码":"证券代码", '截止日期':"统计年度"}, axis=1, inplace=True)
dataset = pd.merge(dataset, csr, on=["统计年度","证券代码"],  how="outer")
# 数字化供应链合作伙伴数量（基于企业年报数据）
supply_network = pd.read_excel("供应链网络关系指标表.xlsx" , skiprows=[0, 2], dtype={'股票代码': 'object'}, parse_dates=['统计截止日期'], engine="openpyxl")
supply_network = supply_network[(supply_network["报表类型"]==1)&(supply_network["统计截止日期"].dt.month==12)]
supply_network['截止日期'] = supply_network['统计截止日期'].dt.year.astype('int')
supply_network = supply_network.groupby(["股票代码", "截止日期"]).count().reset_index()
supply_network = supply_network[["股票代码","截止日期","统计截止日期"]]
# 数据量太少了吧
supply_network.rename({"股票代码":"证券代码", '截止日期':"统计年度", "统计截止日期":"供应链合作伙伴数量"}, axis=1, inplace=True)
dataset = pd.merge(dataset, supply_network, on=["证券代码", "统计年度"],  how="outer")
dataset = dataset[(dataset["统计年度"]>2013)&(dataset["统计年度"]<2024)]
dataset.rename({"证券代码":"股票代码", "统计年度":'截止日期'}, axis=1, inplace=True)
dataset.reset_index(inplace=True, drop=True)
dataset.to_csv("../补充数据.csv", index=False)